In [8]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np

In [9]:
#Set URL and make a request
url = "https://timesofindia.indiatimes.com/world"
r = requests.get(url)

In [11]:
soup = BeautifulSoup(r.content, 'html.parser') 
table = soup.findAll('a', attrs = {'class':'w_img'})

In [12]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

In [14]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [15]:
dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,Trump and Putin issue rare joint statement pro...,Reuters photo\n\nDownload The Times of India N...,Reuters photoDownload The Times of India News ...,"[issue, russian, issued, promoting, russia, in..."
1,Covid-19 pandemic is far from over: WHO,"Apr 28, 2020, 05:08PM IST\n\nSource: AP\n\nThe...","Apr 28, 2020, 05:08PM ISTSource: APThe head of...","[tedros, countries, coronavirus, europe, far, ..."
2,Covid-19 in UK: We are beginning to turn the t...,"Apr 28, 2020, 05:06PM IST\n\nSource: AP\n\nPri...","Apr 28, 2020, 05:06PM ISTSource: APPrime Minis...","[understood, weeks, beginning, johnson, life, ..."
3,US: First responders return to work after Covi...,"Apr 28, 2020, 05:04PM IST\n\nSource: AP\n\nAcr...","Apr 28, 2020, 05:04PM ISTSource: APAcross the ...","[responders, worries, thinks, contracted, work..."
4,Covid-19: Testing won't 'be a problem' for reo...,"Apr 28, 2020, 08:40AM IST\n\nSource: AP\n\nThe...","Apr 28, 2020, 08:40AM ISTSource: APThe White H...","[house, president, getting, problem, white, te..."


## Check Virality of news

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [20]:
FILEPATH="OnlineNewsPopularity\OnlineNewsPopularity.csv"

In [21]:
def clean_cols(data):
   
    clean_col_map = {x: x.lower().strip() for x in list(data)}
    return data.rename(index=str, columns=clean_col_map)

def TrainTestSplit(X, Y, R=0, test_size=0.2):
   
    return train_test_split(X, Y, test_size=test_size, random_state=R)

In [22]:
full_data = clean_cols(pd.read_csv(FILEPATH))
train_set, test_set = train_test_split(full_data, test_size=0.20, random_state=42)

x_train = train_set.drop(['url','shares', 'timedelta', 'lda_00','lda_01','lda_02','lda_03','lda_04','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'timedelta', 'num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_test = test_set['shares']

In [23]:
clf = RandomForestRegressor(random_state=42)
clf.fit(x_train, y_train)

c:\users\karan\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [24]:
rf_res = pd.DataFrame(clf.predict(x_train),list(y_train))

In [25]:
rf_res.reset_index(level=0, inplace=True)
rf_res_df = rf_res.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
rf_res_df.head()

,Actual shares,Predicted shares
0,16100,11492.9
1,508,1334.8
2,1300,1280.0
3,3100,2940.0
4,6900,5960.0
